In [2]:
!pip install cryptography
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature
from pathlib import Path
import os
from cryptography.hazmat.backends import default_backend
import requests
import datetime
# Cell 1 — load KALSHI keys from .env in the same directory as this notebook


# --- Try to use python-dotenv if it's installed (recommended) ---
try:
    from dotenv import load_dotenv  # pip install python-dotenv
    env_path = Path.cwd() / ".env"   # assumes notebook is run from the same directory as .env
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")
    load_dotenv(dotenv_path=env_path, override=False)
except ImportError:
    # --- Fallback: minimal .env parser (no extra install needed) ---
    env_path = Path.cwd() / ".env"
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")

    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        k = k.strip()
        v = v.strip().strip('"').strip("'")
        os.environ.setdefault(k, v)  # don't overwrite if already set

# --- Read your keys into variables ---
public_key = os.getenv("KALSHI-ACCESS-KEY-DEMO")

missing = [k for k, v in {
    "KALSHI-ACCESS-KEY": public_key
}.items() if not v]

if missing:
    raise KeyError(f"Missing env var(s): {missing}. Check your .env formatting and key names.")

print("Loaded Kalshi keys ✅")  # intentionally not printing the secret values


def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key


def sign_pss_text(private_key: rsa.RSAPrivateKey, text: str) -> str:
    message = text.encode('utf-8')
    try:
        signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.DIGEST_LENGTH
            ),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    except InvalidSignature as e:
        raise ValueError("RSA sign PSS failed") from e
    

def make_authenticated_request(path, method='GET', params=None):
    """Make an authenticated request to Kalshi demo API."""
    current_time = datetime.datetime.now()
    timestamp = current_time.timestamp()
    current_time_milliseconds = int(timestamp * 1000)
    timestamp_str = str(current_time_milliseconds)
    
    private_key = load_private_key_from_file('pbhaskarademo.txt')
    
    base_url = 'https://demo-api.kalshi.co'
    
    # Strip query parameters from path before signing
    path_without_query = path.split('?')[0]
    msg_string = timestamp_str + method + path_without_query
    sig = sign_pss_text(private_key, msg_string)
    
    headers = {
        'KALSHI-ACCESS-KEY': public_key,
        'KALSHI-ACCESS-SIGNATURE': sig,
        'KALSHI-ACCESS-TIMESTAMP': timestamp_str
    }
    
    if method == 'GET':
        response = requests.get(base_url + path, headers=headers, params=params)
    else:
        response = requests.post(base_url + path, headers=headers, json=params)
    
    return response

Loaded Kalshi keys ✅


In [ ]:
# Fed & Jerome Powell Markets Analysis

This notebook fetches and analyzes all Kalshi prediction markets related to:
- Federal Reserve (Fed)
- Jerome Powell
- Federal Reserve policy decisions

## Analysis Includes:
- Market category
- Liquidity (trading volume)
- Market duration
- Results with winner information for multi-decision events
- Complete market details and statistics

In [4]:
# ============================================================================
# FETCH FED & JEROME POWELL RELATED MARKETS
# ============================================================================

import time
from datetime import datetime, timedelta, timezone

print("="*80)
print("FETCHING FED & JEROME POWELL RELATED MARKETS")
print("="*80)

# Calculate time window (past year)
now = datetime.now(timezone.utc)
one_year_ago = now - timedelta(days=365)
min_close_ts = int(one_year_ago.timestamp())
max_close_ts = int(now.timestamp())

print(f"\nTime window:")
print(f"  From: {one_year_ago.strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"  To:   {now.strftime('%Y-%m-%d %H:%M:%S UTC')}")

base_url = "https://api.elections.kalshi.com/trade-api/v2"

# Search keywords for Fed/Powell related markets
search_keywords = [
    "fed", "federal reserve", "jerome powell", "powell", 
    "fomc", "federal funds", "interest rate", "fed decision",
    "monetary policy", "fed chair"
]

print(f"\nSearching for markets with keywords: {', '.join(search_keywords)}")
print("\nFetching all series first...")

# Fetch all series
all_series = []
try:
    response = requests.get(f"{base_url}/series", params={"limit": 200})
    if response.status_code == 200:
        all_series = response.json().get('series', [])
        print(f"  ✓ Found {len(all_series)} total series")
    time.sleep(0.5)
except Exception as e:
    print(f"  ✗ Error: {e}")

# Filter series that match our keywords
fed_series = []
for series in all_series:
    title = series.get('title', '').lower()
    ticker = series.get('ticker', '').lower()
    
    for keyword in search_keywords:
        if keyword.lower() in title or keyword.lower() in ticker:
            fed_series.append(series)
            break

print(f"\n✓ Found {len(fed_series)} Fed/Powell related series:")
for s in fed_series[:10]:
    print(f"  - {s['ticker']}: {s['title']}")
if len(fed_series) > 10:
    print(f"  ... and {len(fed_series) - 10} more")

# Fetch all markets from these series
print(f"\nFetching markets from {len(fed_series)} series...")
all_fed_markets = []

for i, series in enumerate(fed_series, 1):
    series_ticker = series['ticker']
    cursor = None
    series_markets = []
    
    while True:
        params = {
            "series_ticker": series_ticker,
            "status": "settled",
            "min_close_ts": min_close_ts,
            "max_close_ts": max_close_ts,
            "limit": 1000
        }
        
        if cursor:
            params["cursor"] = cursor
        
        try:
            response = requests.get(f"{base_url}/markets", params=params)
            
            if response.status_code == 429:
                time.sleep(2)
                continue
            
            if response.status_code != 200:
                break
            
            data = response.json()
            markets = data.get('markets', [])
            series_markets.extend(markets)
            
            cursor = data.get('cursor')
            if not cursor:
                break
            
            time.sleep(0.3)
            
        except:
            break
    
    if series_markets:
        all_fed_markets.extend(series_markets)
        print(f"  [{i}/{len(fed_series)}] {series_ticker}: {len(series_markets)} markets")

print(f"\n{'='*80}")
print(f"✓ TOTAL FED/POWELL MARKETS: {len(all_fed_markets)}")
print(f"{'='*80}")

all_markets = all_fed_markets

FETCHING FED & JEROME POWELL RELATED MARKETS

Time window:
  From: 2025-02-02 00:09:00 UTC
  To:   2026-02-02 00:09:00 UTC

Searching for markets with keywords: fed, federal reserve, jerome powell, powell, fomc, federal funds, interest rate, fed decision, monetary policy, fed chair

Fetching all series first...
  ✓ Found 8222 total series

✓ Found 97 Fed/Powell related series:
  - KXFEDDCEXT: Extending Federal Takeover of DC
  - KXCBDECISIONJAPAN: Bank Of Japan policy interest rate decision
  - KXFEDCONF: Powell confirmation
  - FEDDECISION: Fed meeting
  - FEDCONF: Powell confirmation
  - TAPER: Fed tapering
  - RATEHIKE: Number of Fed rate hikes
  - FEDMEET: Fed emergency meeting
  - RATECUT: Fed rate cut
  - KXLOWESTRATE: Fed funds lowest rate
  ... and 87 more

Fetching markets from 97 series...
  [1/97] KXFEDDCEXT: 1 markets
  [2/97] KXCBDECISIONJAPAN: 12 markets
  [14/97] KXBALANCESHEET: 2 markets
  [15/97] KXPOWELLMENTION: 142 markets
  [16/97] KXFEDGOVNOM: 6 markets
  [17/97] K

In [6]:
# ============================================================================
# ANALYZE AND DISPLAY ALL FED/POWELL MARKETS
# ============================================================================

print("\n" + "="*80)
print("ALL FED & JEROME POWELL MARKETS")
print("="*80)

# Extract volume and create event mapping
markets_with_details = []
event_to_markets = {}

for market in all_markets:
    try:
        # Try to get volume - this is the total traded, not current liquidity
        volume = float(market.get('volume', '0'))
        liquidity = float(market.get('liquidity_dollars', '0'))
        
        # Calculate duration
        open_time = datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
        close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
        duration_days = (close_time - open_time).total_seconds() / (3600 * 24)
        
        event_ticker = market.get('event_ticker', market['ticker'])
        
        # Track markets by event for multi-decision resolution
        if event_ticker not in event_to_markets:
            event_to_markets[event_ticker] = []
        event_to_markets[event_ticker].append(market)
        
        markets_with_details.append({
            'ticker': market['ticker'],
            'title': market.get('title', 'N/A'),
            'category': market.get('category', 'N/A'),
            'volume': volume,
            'liquidity': liquidity,
            'result': market.get('result', 'N/A'),
            'duration_days': duration_days,
            'event_ticker': event_ticker,
            'open_time': open_time,
            'close_time': close_time
        })
    except:
        continue

# Sort by volume descending (not liquidity)
markets_with_details.sort(key=lambda x: x['volume'], reverse=True)

# Display all markets
print(f"\nTotal markets found: {len(markets_with_details)}")
print("\n")

for i, market in enumerate(markets_with_details, 1):
    print(f"\n{'='*100}")
    print(f"Market #{i}")
    print(f"{'='*100}")
    print(f"  Category:     {market['category']}")
    print(f"  Ticker:       {market['ticker']}")
    print(f"  Title:        {market['title']}")
    print(f"  Volume:       ${market['volume']:,.2f}")
    print(f"  Liquidity:    ${market['liquidity']:,.2f}")
    print(f"  Duration:     {market['duration_days']:.1f} days")
    print(f"  Open Date:    {market['open_time'].strftime('%Y-%m-%d')}")
    print(f"  Close Date:   {market['close_time'].strftime('%Y-%m-%d')}")
    
    # Enhanced result display for multi-decision markets
    result_str = market['result'].upper()
    
    # Check if this is part of a multi-decision event
    event_markets = event_to_markets.get(market['event_ticker'], [])
    if len(event_markets) > 1:
        # Find the winning market in this event
        winners = [m for m in event_markets if m.get('result') == 'yes']
        
        if winners:
            if market['result'] == 'yes':
                result_str = f"YES (This option won)"
            else:
                # Show which option actually won
                winner_tickers = [w['ticker'] for w in winners]
                result_str = f"NO (Winner(s): {', '.join(winner_tickers[:3])})"
        else:
            result_str = f"{result_str} (Multi-decision event, no clear winner found)"
    
    print(f"  Result:       {result_str}")
    print(f"  Event Type:   {'Multi-decision' if len(event_markets) > 1 else 'Binary'} ({len(event_markets)} option(s))")

print("\n" + "="*100)
print("\nSUMMARY STATISTICS")
print("="*100)
print(f"  Total markets:     {len(markets_with_details):,}")
print(f"  Average volume:    ${sum(m['volume'] for m in markets_with_details)/len(markets_with_details):,.2f}")
print(f"  Average duration:  {sum(m['duration_days'] for m in markets_with_details)/len(markets_with_details):.1f} days")
print(f"  Total volume:      ${sum(m['volume'] for m in markets_with_details):,.2f}")

# Show category distribution
category_counts = {}
for m in markets_with_details:
    cat = m['category']
    category_counts[cat] = category_counts.get(cat, 0) + 1

print(f"\n  Category Distribution:")
for cat, count in sorted(category_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"    {cat}: {count} markets")

# Show result distribution
result_counts = {}
for m in markets_with_details:
    result = m['result']
    result_counts[result] = result_counts.get(result, 0) + 1

print(f"\n  Result Distribution:")
for result, count in sorted(result_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"    {result}: {count} markets")

# Show multi-decision vs binary breakdown
multi_count = sum(1 for m in markets_with_details if len(event_to_markets.get(m['event_ticker'], [])) > 1)
print(f"\n  Market Types:")
print(f"    Multi-decision: {multi_count}")
print(f"    Binary: {len(markets_with_details) - multi_count}")

# Top 10 by volume
print(f"\n{'='*100}")
print("TOP 10 BY VOLUME")
print("="*100)
for i, m in enumerate(markets_with_details[:10], 1):
    print(f"\n  {i}. {m['ticker']}")
    print(f"     Volume: ${m['volume']:,.2f}")
    print(f"     Title: {m['title']}")

print("="*100)


ALL FED & JEROME POWELL MARKETS

Total markets found: 857



Market #1
  Category:     N/A
  Ticker:       KXFEDDECISION-25SEP-H0
  Title:        Will the Federal Reserve Hike rates by 0bps at their September 2025 meeting?
  Volume:       $42,718,863.00
  Liquidity:    $16,439.15
  Duration:     49.2 days
  Open Date:    2025-07-30
  Close Date:   2025-09-17
  Result:       NO (Winner(s): KXFEDDECISION-25SEP-C25)
  Event Type:   Multi-decision (5 option(s))

Market #2
  Category:     N/A
  Ticker:       KXFEDDECISION-25SEP-C26
  Title:        Will the Federal Reserve Cut rates by >25bps at their September 2025 meeting?
  Volume:       $29,261,331.00
  Liquidity:    $0.00
  Duration:     49.2 days
  Open Date:    2025-07-30
  Close Date:   2025-09-17
  Result:       NO (Winner(s): KXFEDDECISION-25SEP-C25)
  Event Type:   Multi-decision (5 option(s))

Market #3
  Category:     N/A
  Ticker:       KXFEDDECISION-25JUL-C25
  Title:        Will the Federal Reserve Cut rates by 25bps at thei